In [1]:
from ecdsa import SigningKey, SECP256k1, VerifyingKey
from ecdsa.util import randrange_from_seed__trytryagain
import hashlib


In [115]:
from ecdsa import SigningKey, SECP256k1, VerifyingKey
import os
import hashlib

def generate_valid_key_pair():
    private_key = SigningKey.generate(curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    return private_key.to_string().hex(), public_key.to_string().hex()

# Function to compute a shared secret using ECDH
def compute_shared_secret(private_key, public_key):
    shared_secret_point = private_key.privkey.secret_multiplier * public_key.pubkey.point
    return shared_secret_point.x().to_bytes(32, 'big')

# Function to hash a shared secret
def hash_shared_secret(shared_secret):
    return hashlib.sha256(shared_secret).digest()

# Function to generate a stealth address
def generate_stealth_address(spend_pub_key_bytes, view_pub_key_bytes):
    # Generate a random ephemeral private key
    ephemeral_private_key = SigningKey.generate(curve=SECP256k1)

    # Derive the ephemeral public key
    ephemeral_public_key = ephemeral_private_key.get_verifying_key()

    # Compute the shared secret
    view_public_key = VerifyingKey.from_string(view_pub_key_bytes, curve=SECP256k1)
    shared_secret = compute_shared_secret(ephemeral_private_key, view_public_key)

    # Hash the shared secret
    s_h = hash_shared_secret(shared_secret)

    # Extract the view tag 'v'
    view_tag_v = s_h[0]

    # Compute the hashed shared secret times the generator point 'G'
    S_h = int.from_bytes(s_h, 'big') * SECP256k1.generator

    # Compute the recipient’s stealth public key
    spend_public_key = VerifyingKey.from_string(spend_pub_key_bytes, curve=SECP256k1)
    P_stealth = VerifyingKey.from_public_point(spend_public_key.pubkey.point + S_h, curve=SECP256k1)
 
    # Convert the stealth public key to an address
    stealth_address = P_stealth.to_string("uncompressed").hex()

    # Return the stealth address, ephemeral public key, and the view tag
    return stealth_address, ephemeral_public_key.to_string().hex(), view_tag_v

def compute_stealth_private_key(viewing_private_key_hex, ephemeral_public_key_hex, spending_private_key_hex):
    # Convert hex keys to bytes
    viewing_private_key_bytes = bytes.fromhex(viewing_private_key_hex)
    spending_private_key_bytes = bytes.fromhex(spending_private_key_hex)
    ephemeral_public_key_bytes = bytes.fromhex(ephemeral_public_key_hex)
    
    # Create signing and verifying key objects from bytes
    viewing_private_key = SigningKey.from_string(viewing_private_key_bytes, curve=SECP256k1)
    ephemeral_public_key = VerifyingKey.from_string(ephemeral_public_key_bytes, curve=SECP256k1)
    
    # Compute the shared secret using ECDH
    shared_secret = compute_shared_secret(viewing_private_key, ephemeral_public_key)
    
    # Hash the shared secret
    s_h = hash_shared_secret(shared_secret)
    
    # Compute the stealth private key
    s_h_int = int.from_bytes(s_h, 'big')
    spending_private_key_int = int.from_bytes(spending_private_key_bytes, 'big')
    
    # Calculate the stealth private key
    stealth_private_key_int = (spending_private_key_int + s_h_int) % SECP256k1.order
    stealth_private_key = stealth_private_key_int.to_bytes(32, byteorder='big')
    
    return stealth_private_key.hex()

def regenerate_stealth_address(stealth_private_key_hex):
    # Convert the hex private key to a byte string
    stealth_private_key_bytes = bytes.fromhex(stealth_private_key_hex)
    
    # Create the SigningKey object from the private key byte string
    stealth_private_key = SigningKey.from_string(stealth_private_key_bytes, curve=SECP256k1)
    
    # Get the corresponding public key
    stealth_public_key = stealth_private_key.get_verifying_key()
    
    # Encode the public key to its compressed representation
    stealth_address = stealth_public_key.to_string("compressed").hex()
    return stealth_address

# Generate a valid pair of keys for testing
spending_private_key_hex, spending_pub_key_hex = generate_valid_key_pair()
viewing_private_key_hex, viewing_pub_key_hex = generate_valid_key_pair()

# Generate stealth address using the provided meta-address keys
spend_pub_key_bytes = bytes.fromhex(spending_pub_key_hex)
view_pub_key_bytes = bytes.fromhex(viewing_pub_key_hex)
stealth_address, ephemeral_public_key_hex, view_tag = generate_stealth_address(spend_pub_key_bytes, view_pub_key_bytes)

stealth_private_key_hex = compute_stealth_private_key(viewing_private_key_hex, ephemeral_public_key_hex, spending_private_key_hex)

regenerated_stealth_address = regenerate_stealth_address(stealth_private_key_hex)

print("Spending Private Key (hex):", spending_private_key_hex)
print("Spending Public Key (hex):", spending_pub_key_hex)
print("Viewing Private Key (hex):", viewing_private_key_hex)
print("Viewing Public Key (hex):", viewing_pub_key_hex)
print("Stealth Address (hex):", stealth_address)
print("Ephemeral Public Key (hex):", ephemeral_public_key_hex)
print("View Tag (hex):", format(view_tag, '02x'))
print("Stealth Private Key (hex):", stealth_private_key_hex)
print("Regenerated Stealth Address (hex):", regenerated_stealth_address)


Spending Private Key (hex): 618b7e591ba109f8fbb05fa7c27af91fdd7143dec9bffe6734d6cfc90b3bfdba
Spending Public Key (hex): b9c085943ebfc9d0b0b4b0e6038893171e03302ac488cf129e3c6ca9a3936244d6f4b7d06e85a4ffc4496d6c31e097bcd399a3d9fbfa5ba9329410e20743cf88
Viewing Private Key (hex): 225a623dafb17a567d72444ed86a836125dc6d5bb79ea7a6b00d4591942f7016
Viewing Public Key (hex): daa8332983febed8135235eeb3f06d0526439f2f63d23135fd38ce8976fd4d52171859626a205d266074e186be33862dc6e6173dcd74c842a56410cf068bb1de
Stealth Address (hex): 043fd60dc90212a11e3d360caf593a2daa990348068092ee12370e4c1f494e21cbd50da1b8ebad972b1f1f5f4a90d8044c2de40a83645b1a7c19ec8db7cfd5aa31
Ephemeral Public Key (hex): 04f60eee3ead24379036a6a0bef0c626622f1d5ac81dfe2b2525fb4b1b165d0a33f64093dc4546bc987a9a482a5a6e25d3af5e453120b795766242758d400d0a
View Tag (hex): 06
Stealth Private Key (hex): 67a1ca58c99c5583fcf8cfa9463db1568f1edd87e5acd42715312233e72a3f04
Regenerated Stealth Address (hex): 033fd60dc90212a11e3d360caf593a2daa990348068092e

In [116]:
from Crypto.Hash import keccak

def private_key_to_ethereum_address(private_key_hex):
    # Convert the hex private key to bytes
    private_key_bytes = bytes.fromhex(private_key_hex)
    
    # Derive the ECDSA public key
    signing_key = SigningKey.from_string(private_key_bytes, curve=SECP256k1)
    public_key = signing_key.get_verifying_key().to_string("uncompressed")[1:]
    
    # Hash the public key using Keccak-256
    keccak_hash = keccak.new(digest_bits=256)
    keccak_hash.update(public_key)
    hash_digest = keccak_hash.digest()
    
    # Take the last 20 bytes for the Ethereum address
    ethereum_address = '0x' + hash_digest[-20:].hex()
    
    return ethereum_address

# Stealth Private Key (hex)
stealth_private_key_hex = stealth_private_key_hex

# Derive the Ethereum address from the stealth private key
ethereum_address = private_key_to_ethereum_address(stealth_private_key_hex)
print("Ethereum Address:", ethereum_address)

Ethereum Address: 0x67c2a76e3e92d47cc76fd485b963d315ad09f878


In [117]:
from Crypto.Hash import keccak

def public_key_to_eth_address(public_key_hex):
    # Ensure the public key is in the correct format (remove '04' if present for uncompressed keys)
    if public_key_hex.startswith('04'):
        public_key_hex = public_key_hex[2:]
        
    public_key_bytes = bytes.fromhex(public_key_hex)
    
    # Apply the Keccak-256 hash function to the public key
    keccak_digest = keccak.new(digest_bits=256)
    keccak_digest.update(public_key_bytes)
    
    # Ethereum address is the last 20 bytes of the hash
    eth_address = '0x' + keccak_digest.hexdigest()[-40:]
    return eth_address

# Your stealth address (public key in hex format)
stealth_address_hex = stealth_address

# Convert to Ethereum address
ethereum_address = public_key_to_eth_address(stealth_address_hex)
print("Ethereum Address:", ethereum_address)

Ethereum Address: 0x67c2a76e3e92d47cc76fd485b963d315ad09f878
